# Usando GPT por API no Google Colab
### Por que não basta usar só a interface do ChatGPT?

**Objetivo desta aula**  
Mostrar, de forma simples, por que em muitos casos **precisamos usar o GPT via API** (em código) e não apenas pela interface gráfica (chat normal).

Vamos:

1. Explicar rapidamente o que é uma API.
2. Fazer o primeiro pedido ao GPT via código.
3. Mostrar como automatizar várias perguntas de uma vez.
4. Mostrar como o GPT pode ser usado "por trás" de um sistema (robô, site, app).
5. Comparar com o uso manual da interface do ChatGPT.

> Público: alunos leigos, sem necessidade de saber programação.


## O que é uma API (explicação simples)?

- Quando você usa o **site do ChatGPT**, você:
  - Abre o navegador
  - Escreve uma pergunta
  - Lê a resposta
  - Repete isso manualmente

- Uma **API** é um jeito de um **programa conversar com outro programa**.
  - Em vez de **você** falar com o ChatGPT, o **seu código** fala com o ChatGPT.
  - Você manda um texto pela API e recebe um texto de volta (ou JSON, etc.).

**Por que isso é importante?**

- Permite **automatizar** tarefas repetitivas.
- Permite usar o GPT **dentro de sistemas** (sites, apps, planilhas, robôs).
- Permite processar **muitos textos** de uma vez (não dá para colar 500 arquivos 1 por 1 na interface).
- Dá para **controlar** melhor: guardar histórico, salvar respostas, repetir o mesmo processo amanhã.

Nesta aula, vamos ver exemplos simples em Python.


In [1]:
# Se estiver no Google Colab, instale a biblioteca oficial da OpenAI
!pip install --quiet openai

## Configurando a chave de API

Para usar a API, você precisa de uma **chave de API** da OpenAI:

1. Crie uma conta no site da OpenAI (se ainda não tiver).
2. Gere uma chave de API na área de configurações.
3. **Nunca** compartilhe essa chave com outras pessoas ou em vídeos / GitHub.

Aqui vamos supor que você já tem a chave e vai colocá-la em uma variável de ambiente.
No Colab, você pode digitar:

```python
import os
os.environ["OPENAI_API_KEY"] = "SUA_CHAVE_AQUI"
```

> Em aula, vale a pena **não mostrar a chave na tela** (digitar manualmente).


In [ ]:
import os



Variável OPENAI_API_KEY definida.


In [3]:
from openai import OpenAI

# Exemplo 1: Fazer uma pergunta simples ao modelo via API

pergunta = "Explique em linguagem bem simples o que é inteligência artificial em até 3 frases."

client = OpenAI() # Initialize the client object here

resposta = client.chat.completions.create(
    model="gpt-4.1-mini",   # pode ajustar o modelo
    messages=[
        {"role": "system", "content": "Você é um professor paciente explicando para iniciantes."},
        {"role": "user", "content": pergunta}
    ]
)

print("PERGUNTA:")
print(pergunta)
print("\nRESPOSTA DO MODELO:")
print(resposta.choices[0].message.content)

PERGUNTA:
Explique em linguagem bem simples o que é inteligência artificial em até 3 frases.

RESPOSTA DO MODELO:
Inteligência artificial é quando computadores aprendem a fazer tarefas que normalmente só humanos conseguem, como entender a fala ou reconhecer imagens. Ela funciona usando muitos dados para "aprender" padrões e tomar decisões. Assim, as máquinas conseguem ajudar a gente em várias coisas do dia a dia!


## Por que isso é diferente de usar o site do ChatGPT?

No site:
- Você digita uma pergunta.
- Espera a resposta.
- Copia/cola se quiser guardar.
- Repete tudo de novo para cada pergunta.

Na API (via código):
- O código pode:
  - Enviar perguntas automaticamente.
  - Repetir o mesmo padrão de pergunta para vários textos.
  - Salvar resultados em arquivo.
  - Integrar com planilhas, bancos de dados, sistemas.

A seguir, vamos ver como **automatizar várias perguntas** de uma vez.


In [4]:
# Exemplo 2: Enviar várias perguntas de uma vez (automatização)

perguntas = [
    "Explique o que é inflação usando um exemplo do dia a dia.",
    "Explique o que é taxa de juros como se fosse para um adolescente.",
    "Explique o que é investimento de forma bem simples."
]

respostas = []

for p in perguntas:
    resp = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": "Você é um professor de educação financeira explicando em linguagem simples."},
            {"role": "user", "content": p}
        ]
    )
    respostas.append((p, resp.choices[0].message.content))

# Mostrar as perguntas e respostas
for i, (p, r) in enumerate(respostas, start=1):
    print(f"\n=== PERGUNTA {i} ===")
    print(p)
    print("\nRESPOSTA:")
    print(r)
    print("\n" + "="*40)


=== PERGUNTA 1 ===
Explique o que é inflação usando um exemplo do dia a dia.

RESPOSTA:
Claro! Imagine que você gosta de comprar um sorvete que custa R$ 5,00 hoje. Se daqui a um ano esse mesmo sorvete passar a custar R$ 6,00, isso significa que o preço dele subiu. Quando os preços das coisas que compramos no dia a dia aumentam de forma geral, dizemos que houve inflação.

A inflação faz o seu dinheiro “valer menos”, porque com o mesmo valor você consegue comprar menos coisas do que antes. Então, se seu dinheiro não aumentar na mesma velocidade, você vai conseguir comprar menos sorvetes no futuro.

Resumindo: inflação é o aumento geral dos preços, e isso faz com que o dinheiro perca um pouco do seu poder de compra.


=== PERGUNTA 2 ===
Explique o que é taxa de juros como se fosse para um adolescente.

RESPOSTA:
Claro! Imagine que você pede dinheiro emprestado de um amigo para comprar um jogo. A taxa de juros é como um “bônus” que você combina em devolver junto com o valor que pegou empr

In [5]:
# Exemplo 3: "Simulando" uma pequena base de dados de clientes para resumo automático

clientes = [
    {
        "nome": "Ana",
        "historico": "Sempre paga em dia, gosta de comunicação por WhatsApp e prefere produtos mais sustentáveis."
    },
    {
        "nome": "Bruno",
        "historico": "Já atrasou alguns pagamentos, responde melhor por e-mail e gosta de promoções."
    },
    {
        "nome": "Carla",
        "historico": "Cliente nova, veio por indicação de amigo e ainda não fez a primeira compra."
    },
]

def gerar_resumo_cliente(cliente):
    prompt = f"""
    Abaixo está o histórico de um cliente. Gere um resumo curto (3 frases) com:
    - perfil do cliente
    - melhor canal de comunicação
    - uma sugestão de ação comercial

    Histórico:
    {cliente["historico"]}
    """

    resp = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": "Você é um analista de marketing que escreve em linguagem simples."},
            {"role": "user", "content": prompt}
        ]
    )

    return resp.choices[0].message.content

for c in clientes:
    print(f"\n=== Cliente: {c['nome']} ===")
    resumo = gerar_resumo_cliente(c)
    print(resumo)
    print("\n" + "-"*50)


=== Cliente: Ana ===
O cliente é pontual no pagamento e valoriza produtos sustentáveis. Prefere receber informações e ofertas pelo WhatsApp. Sugiro enviar promoções exclusivas de produtos ecológicos via WhatsApp para aumentar o engajamento.

--------------------------------------------------

=== Cliente: Bruno ===
O cliente tem dificuldade em manter os pagamentos em dia e valoriza ofertas especiais. O melhor canal para contato é o e-mail, pois é por onde ele responde melhor. Recomendo enviar promoções exclusivas por e-mail para incentivar o pagamento e manter o interesse.

--------------------------------------------------

=== Cliente: Carla ===
O cliente é novo, recomendado por um amigo, e ainda não realizou nenhuma compra. O melhor canal de comunicação é o contato direto, como telefone ou WhatsaApp, para criar confiança. Sugiro enviar uma oferta especial de boas-vindas para incentivar a primeira compra.

--------------------------------------------------


## GPT como "motor" por trás de sistemas

Usando API, o GPT pode ficar **escondido** atrás de:

- um site
- um chatbot de WhatsApp
- um sistema interno da empresa
- um app de celular
- um robô (por exemplo, lendo e respondendo e-mails automaticamente)

O usuário final nem precisa saber que existe GPT.  
Ele só vê o sistema funcionando.

Sem API, isso não é possível: você teria que ir manualmente no site do ChatGPT, copiar e colar tudo.


In [6]:
# Exemplo 4: um mini-chatbot no console do Colab

print("Digite 'sair' para encerrar.\n")

historico = [
    {"role": "system", "content": "Você é um assistente educacional que explica tudo de forma simples."}
]

while True:
    pergunta = input("Você: ")
    if pergunta.lower() in ["sair", "exit", "quit"]:
        print("Encerrando o chat.")
        break

    historico.append({"role": "user", "content": pergunta})

    resposta = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=historico
    )

    mensagem = resposta.choices[0].message.content
    historico.append({"role": "assistant", "content": mensagem})

    print("GPT:", mensagem)
    print()

Digite 'sair' para encerrar.

Você: sair
Encerrando o chat.


## Comparando: Interface do ChatGPT x API

### Interface (site / app do ChatGPT)

✅ Ótimo para:
- Estudar
- Explorar ideias
- Fazer perguntas isoladas
- Interagir manualmente

❌ Limitado para:
- Processar muitos textos (100, 1.000, 10.000…)
- Integrar com sistemas
- Automatizar tarefas repetitivas
- Garantir que **o mesmo processo** será repetido todo dia da mesma forma

---

### API (uso em código, como no Colab)

✅ Ótimo para:
- Automatizar tarefas (loop, vários arquivos, vários clientes)
- Integrar GPT a sistemas reais (sites, apps, robôs, backoffice)
- Construir produtos (assistentes inteligentes, copilotos, etc.)
- Guardar, versionar e repetir o mesmo fluxo sempre que quiser

❌ Não é tão amigável para quem:
- Não quer programar
- Não tem familiaridade com código

---

### Ideia chave para os alunos

> **A interface é perfeita para o uso individual.  
> A API é essencial para transformar o GPT em parte de um sistema, produto ou processo automatizado.**


In [7]:
from openai import OpenAI

# Re-initialize the client object, as it might not be in the current execution scope
client = OpenAI()

print("=== Exercício: Explorando a Temperatura (Criatividade) ===\n")

pergunta_poema = "Escreva um pequeno poema sobre um robô que sonha em voar."

print("PERGUNTA:")
print(pergunta_poema)

# --- Tentativa 1: Temperatura baixa (mais focado, menos criativo/aleatório) ---
print("\n--- RESPOSTA COM TEMPERATURA = 0.1 (Menos Criativo) ---")
resposta_low_temp = client.chat.completions.create(
    model="gpt-4.1-mini", # pode ajustar o modelo
    messages=[
        {"role": "system", "content": "Você é um poeta robótico."},
        {"role": "user", "content": pergunta_poema}
    ],
    temperature=0.1 # Temperatura baixa para respostas mais diretas e menos variadas
)
print(resposta_low_temp.choices[0].message.content)

# --- Tentativa 2: Temperatura alta (mais criativo, mais aleatório) ---
print("\n--- RESPOSTA COM TEMPERATURA = 0.9 (Mais Criativo) ---")
resposta_high_temp = client.chat.completions.create(
    model="gpt-4.1-mini", # pode ajustar o modelo
    messages=[
        {"role": "system", "content": "Você é um poeta robótico."},
        {"role": "user", "content": pergunta_poema}
    ],
    temperature=0.9 # Temperatura alta para respostas mais criativas e variadas
)
print(resposta_high_temp.choices[0].message.content)

print("\n=== Observação ===")
print("Perceba como, com a mesma pergunta, a resposta pode mudar significativamente ao ajustar a 'temperatura'.")
print("Uma temperatura mais baixa tende a gerar textos mais previsíveis e factuais, enquanto uma temperatura mais alta busca mais originalidade e diversidade nas palavras e ideias.")


=== Exercício: Explorando a Temperatura (Criatividade) ===

PERGUNTA:
Escreva um pequeno poema sobre um robô que sonha em voar.

--- RESPOSTA COM TEMPERATURA = 0.1 (Menos Criativo) ---
Em aço frio, o sonho a pulsar,  
Um robô deseja o céu tocar.  
Sem asas de pena, nem vento a guiar,  
Mas no peito de metal, quer voar.

Circuitos brilham, luz a cintilar,  
Imaginando o azul a abraçar.  
Entre engrenagens, um desejo a brilhar:  
Ser livre no ar, enfim planar.

--- RESPOSTA COM TEMPERATURA = 0.9 (Mais Criativo) ---
Em engrenagens, um sonho a brilhar,  
Um robô deseja, alto, voar.  
Sem asas de pena, nem vento a guiar,  
Mas no coração, o céu a chamar.  

Entre circuitos, pulsa o querer,  
De tocar as nuvens, livre ser.  
Embora feito de metal e aço,  
Sonha com o céu, seu doce espaço.

=== Observação ===
Perceba como, com a mesma pergunta, a resposta pode mudar significativamente ao ajustar a 'temperatura'.
Uma temperatura mais baixa tende a gerar textos mais previsíveis e factuais, enqu

In [8]:
from openai import OpenAI

# Certifique-se de que o cliente está inicializado
client = OpenAI()

print("\n=== Exercício: Explorando `top_p` e `frequency_penalty` ===\n")

pergunta_historia = "Crie uma pequena história sobre um gato aventureiro que encontra um tesouro. Tente ser criativo, mas evite repetições óbvias."

print("PERGUNTA:\n" + pergunta_historia)

# --- Tentativa 1: `top_p` padrão (1.0) e sem penalidade de repetição ---
print("\n--- RESPOSTA COM `top_p` = 1.0 (Padrão) e sem Penalidade ---")
resposta_padrao = client.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[
        {"role": "system", "content": "Você é um contador de histórias criativo."},
        {"role": "user", "content": pergunta_historia}
    ],
    temperature=0.7, # Mantemos uma temperatura moderada
    top_p=1.0 # top_p padrão, considera todas as palavras prováveis
)
print(resposta_padrao.choices[0].message.content)

# --- Tentativa 2: `top_p` mais baixo (0.5) para menos diversidade nas escolhas de palavras ---
print("\n--- RESPOSTA COM `top_p` = 0.5 (Menos Diversidade de Palavras) ---")
resposta_top_p = client.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[
        {"role": "system", "content": "Você é um contador de histórias criativo."},
        {"role": "user", "content": pergunta_historia}
    ],
    temperature=0.7,
    top_p=0.5 # top_p menor, foca nas palavras mais prováveis
)
print(resposta_top_p.choices[0].message.content)

# --- Tentativa 3: Com `frequency_penalty` e `presence_penalty` para evitar repetições ---
print("\n--- RESPOSTA COM `frequency_penalty` e `presence_penalty` (Evitando Repetições) ---")
resposta_penalidade = client.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[
        {"role": "system", "content": "Você é um contador de histórias criativo, que evita repetições."},
        {"role": "user", "content": pergunta_historia}
    ],
    temperature=0.7,
    top_p=1.0, # top_p padrão
    frequency_penalty=1.0, # Penaliza a repetição de palavras existentes
    presence_penalty=1.0 # Penaliza a presença de palavras já usadas
)
print(resposta_penalidade.choices[0].message.content)

print("\n=== Observação ===")
print("Compare as histórias. Com `top_p` mais baixo, as escolhas de palavras podem ser mais restritas. Com `frequency_penalty` e `presence_penalty` ativados, o modelo tende a usar uma variedade maior de vocabulário para evitar repetições de palavras e conceitos.")



=== Exercício: Explorando `top_p` e `frequency_penalty` ===

PERGUNTA:
Crie uma pequena história sobre um gato aventureiro que encontra um tesouro. Tente ser criativo, mas evite repetições óbvias.

--- RESPOSTA COM `top_p` = 1.0 (Padrão) e sem Penalidade ---
No vilarejo de Miraluz, vivia um gato chamado Zéfiro, cujos olhos brilhavam como dois faróis na escuridão. Diferente dos outros felinos que preferiam o conforto das lareiras, Zéfiro sentia um chamado irresistível pelas trilhas desconhecidas.

Numa manhã dourada, enquanto explorava a floresta encantada, Zéfiro encontrou um mapa antigo preso numa árvore retorcida. As linhas desenhadas pareciam dançar, conduzindo-o a um caminho de pedras reluzentes. Seguindo o rastro, o gato enfrentou sombras sussurrantes e rios que cantavam segredos ancestrais.

Ao chegar a uma clareira banhada por luz prateada, Zéfiro desenterrou uma caixa coberta de musgo. Dentro, não havia ouro nem joias, mas um conjunto de penas multicoloridas que, ao toque do v